## Importación librerias y dataset

In [146]:
import pandas as pd
import numpy as np

import matplotlib as plt
import seaborn as sns

In [173]:
df=pd.read_csv('data\players_total2.csv')

In [148]:
definiciones=pd.read_csv('data\definiciones_fifa.csv')

In [149]:
definiciones[definiciones['campo']=='ram']

,campo,descripcion
88,ram,player attribute playing as Right Attacking Mi...


In [150]:
definiciones['descripcion'].unique()

array(['unique player ID on sofifa', 'URL of the scraped player',
       'player short name', 'player long name',
       'player preferred positions', 'player current overall attribute',
       'player potential overall attribute', 'player value (in EUR)',
       'player weekly wage (in EUR)', 'player age',
       'player date of birth', 'player height (in cm)',
       'player weight (in kg)', 'club id',
       'club name where the player plays', 'league name of the club', nan,
       'player jersey number in the club',
       'club loaning out the player - if applicable',
       'date when the player joined his current club',
       'player contract expiration date', 'player nationality id',
       'player nationality', 'nation team id',
       'player position in the national team',
       'player jersey number in the national team',
       'player preferred foot', 'player weak foot attribute',
       'player skill moves attribute',
       'player international reputation attribute',

In [151]:
definiciones['campo'].unique()

array(['sofifa_id', 'player_url', 'short_name', 'long_name',
       'player_positions', 'overall', 'potential', 'value_eur',
       'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id',
       'club_name', 'league_name',
       'league_level,"league rank of the club (e.g. English Premier League is 1, English League Championship is 2, etc.)"',
       'club_position,"player position in the club (e.g. SUB means substitute, RES means reserve)"',
       'club_jersey_number', 'club_loaned_from', 'club_joined',
       'club_contract_valid_until', 'nationality_id', 'nationality_name',
       'nation_team_id', 'nation_position', 'nation_jersey_number',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'work_rate', 'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'player_traits', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_he

## Porteros

In [152]:
porteros=df[df['player_positions']=='GK']

In [153]:
#me quedo con el resto de jugadores
df=df[df['player_positions']!='GK']

## Defensas

In [154]:
#"player position in the club (e.g. SUB means substitute, RES means reserve)"
df.club_position.unique().tolist()

['CF',
 'LW',
 'SUB',
 'ST',
 'LCM',
 'RES',
 'RS',
 'LM',
 'RCM',
 'LCB',
 'CAM',
 'RW',
 'LS',
 'LDM',
 'RCB',
 'CB',
 'CDM',
 'CM',
 'RDM',
 'RB',
 'RM',
 'LB',
 'RAM',
 'LAM',
 nan,
 'RWB',
 'LF',
 'RF',
 'LWB']

In [155]:
pos_defensas=['LCB','RCB','CB','RB','LB','RWB','LWB']

In [156]:
isdefense=df.club_position.isin(pos_defensas)
defensas=df[isdefense]

In [157]:
#me quedo con el resto de jugadores
isnotdefense=~df.club_position.isin(pos_defensas)
df=df[isnotdefense]

## Centrocampistas

In [158]:
df.club_position.unique().tolist()

['CF',
 'LW',
 'SUB',
 'ST',
 'LCM',
 'RES',
 'RS',
 'LM',
 'RCM',
 'CAM',
 'RW',
 'LS',
 'LDM',
 'CDM',
 'CM',
 'RDM',
 'RM',
 'RAM',
 'LAM',
 nan,
 'LF',
 'RF']

In [159]:
pos_medios=['LW','LCM','RCM','CAM','RW','LDM','CDM','CM','RDM','RM','RAM','LAM','LM']

In [160]:
ismidfilder=df.club_position.isin(pos_medios)
medios=df[ismidfilder]

In [161]:
#me quedo con el resto de jugadores
isnotmidfilder=~df.club_position.isin(pos_medios)
df=df[isnotmidfilder]

## Delanteros

In [162]:
df.club_position.unique().tolist()

['CF', 'SUB', 'ST', 'RES', 'RS', 'LS', nan, 'LF', 'RF']

In [163]:
pos_delanteros=['CF','ST','RS','LS','RF']

In [164]:
isforward=df.club_position.isin(pos_delanteros)
delanteros=df[isforward]

In [165]:
#me quedo con el resto de jugadores
isnotforward=~df.club_position.isin(pos_delanteros)
df=df[isnotforward]

## Jugadores pendientes de distribuir

In [166]:
#Estrategia: Hacer función
#Si hay una posición poner el grupo de esa posicion
#Buscar si alguna de las posiciones del jugador coincide con la posicion del jugador en la selección. Si coincide tiene esa posición y buscamos de que grupo es.
#Si no coincide hacemos un conteo de las posiciones que tiene dentro de cada grupo => nos quedamos con el grupo con más
#Si está igualado llamamos al jugador polivalente y lo duplicamos

In [174]:
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [168]:
df['player_positions']=df['player_positions'].apply(lambda x: lista_posicion(x))
    

In [169]:
def posicion_jugador(x) :
    #categorizo las posiciones en diccionario y las agrupo en un diccionario
    dic_posicion = {}
    dic_posicion['delantero'] = 'ls', 'st', 'rs', 'lw', 'lf', 'cf', 'rf', 'rw'
    dic_posicion['centrocampista'] = 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 'rm', 'ldm', 'cdm', 'rdm'
    dic_posicion['defensa'] = 'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb'
    dic_posicion['portero'] = 'gk'
    #extraigo la lista de posiciones
    lista_x = x.replace(' ','').split(',')
    #transforomo la posicion
    for x in lista_x:
        for posicion, items in dic_posicion.items():
            if x.lower() in items:
                return (posicion)
    print('Error')

## Definitivo

In [178]:
#df=pd.read_csv('data\players_total2.csv')

In [179]:
#funcion transformación en lista
def lista_posicion(x):
    return (x.replace(' ','').split(','))

In [180]:
#transforma valores de columna player_positions en una lista
df['player_positions']=df['player_positions'].apply(lambda x: lista_posicion(x))

In [181]:
    
for i in df.index:
            if len(df.at[i,'player_positions'])==1:
                if df.at[i,'player_positions'][0]=='GK':
                    df.at[i,'player_positions']='portero'
                if df.at[i,'player_positions'][0] in pos_defensas:
                    df.at[i,'player_positions']='defensa'
                elif df.at[i,'player_positions'][0] in pos_medios:
                    df.at[i,'player_positions']='centrocampista'
                elif df.at[i,'player_positions'][0] in pos_delanteros:
                    df.at[i,'player_positions']='delantero'
            else:
                if df.at[i,'player_positions'][0] in pos_defensas:
                    df.at[i,'player_positions']='defensa'
                elif df.at[i,'player_positions'][0] in pos_medios:
                    df.at[i,'player_positions']='centrocampista'
                elif df.at[i,'player_positions'][0] in pos_delanteros:
                    df.at[i,'player_positions']='delantero'
                elif df.at[i,'player_positions'][0]=='GK':
                    df.at[i,'player_positions']='portero'
                    

In [182]:
df[(df['player_positions']!='defensa')&(df['player_positions']!='centrocampista')&(df['player_positions']!='delantero')&(df['player_positions']!='portero')]

,Unnamed: 0,Unnamed: 0.1,sofifa_id,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,...,rdm,rwb,lb,lcb,cb,rcb,rb,gk,año_version,progresion_anual


In [185]:
df.groupby('player_positions')['sofifa_id'].count()

player_positions
centrocampista    36424
defensa           29039
delantero         12650
portero            9634
Name: sofifa_id, dtype: int64

In [186]:
df['ls']

0        89+3
1        91+1
2        84+3
3        87+3
4        38+3
         ... 
87742    46+2
87743    34+2
87744    32+2
87745    46+1
87746    47+2
Name: ls, Length: 87747, dtype: object